In [1]:
%run segment_module.py
import os
import networkx as nx
METHOD = cv2.TM_SQDIFF
PATH_TO_CHARS = r'C:\Users\BDG3CLJ\Desktop\OpenCV_tutorials\CARACTERE' #Path to all images
CLUSTERS = r'C:\Users\BDG3CLJ\Desktop\OpenCV_tutorials\CLUSTERS' #Path to resulted clusters
NUM_CLUST = 32 # number of clusters

In [2]:
def distance(img1, img2, method):
    """
    Function to calculate the distance between two images using a given method
    :param: img1 - first image (np array with shape (h, w)) - grayscale
    :param: img2 - second image (np array with shape (h, w)) - grayscale
    :method: distance function
    """
    return cv2.matchTemplate(img1, img2, method)[0][0]

In [30]:
"""
Cell for collecting all images
"""
img_lst = []
i = 0
for img_dir in os.listdir(PATH_TO_CHARS):
    if i > 19:
        break
    print(img_dir)
    path = PATH_TO_CHARS + "\\" + img_dir
    _, imgs, _ = get_files(path)
    for img in imgs:
        name, im = read_image(img)
        img_lst.append([path, name, im])
    i += 1

e_p01_ebs3-1-label-benz2-056-nok.jpg
e_p01_ebs3-1-label-benz2-057-nok.jpg
e_p01_ebs3-1-label-benz2-058-nok.jpg
e_p01_ebs3-1-label-ford-02-015-nok.jpg
e_p01_ebs3-1-label-ford-02-016-nok.jpg
e_p01_ebs3-1-label-ford-02-017-nok.jpg
e_p01_ebs3-1-label-ford-02-018-nok.jpg
e_p01_ebs3-1-label-ford-02-021-nok.jpg
e_p01_ebs3-1-label-ford-02-022-nok.jpg
e_p01_ebs3-1-label-ford-02-023-nok.jpg
e_p01_ebs3-1-label-ford-02-024-nok.jpg
e_p01_ebs3-1-label-ford-02-027-nok.jpg
e_p01_ebs3-1-label-ford-02-033-nok.jpg
e_p01_ebs3-1-label-ford-02-036-nok.jpg
e_p01_ebs3-1-label-ford-02-037-nok.jpg
e_p01_ebs3-1-label-ford-02-038-nok.jpg
e_p01_ebs3-1-label-ford-02-039-nok.jpg
e_p01_ebs3-1-label-ford-02-044-nok.jpg
e_p01_ebs3-1-label-ford-02-064-nok.jpg
e_p01_ebs3-1-label-ford-02-065-nok.jpg


In [31]:
"""
Constructing the complete graph of images
-weighted graph (i, j, w) - i:(first image), j:(second image), w: (distance between images)
"""
g = nx.Graph()
for i in range(len(img_lst) - 1):
    if i % 1000 == 0:
        print(i)
    for j in range(i + 1, len(img_lst)):
        g.add_edge(img_lst[i][0] + '\\' + img_lst[i][1], img_lst[j][0] + '\\' + img_lst[j][1], weight=(distance(img_lst[i][2], img_lst[j][2], METHOD)))

0
1000


In [32]:
mst = nx.minimum_spanning_tree(g) #computing the minimum spanning tree of the graph

In [33]:
edges = [e for e in mst.edges.data()] #list of mst edges

In [34]:
edges = sorted(edges, key=lambda x : abs(x[2]['weight']), reverse = True) #sorting edges by weight (descending)

In [35]:
to_remove = edges[:NUM_CLUST] #getting the first NUM_CLUST edges

In [36]:
mst.remove_edges_from(to_remove) #removing first NUM_CLUST edges

In [37]:
#Computing the connected components of the mst after removing the fist NUM_CLUST edges 
#Every connected component is a cluster
comp = []
for i, x in enumerate(nx.connected_components(mst)):
    comp.append(x)

In [39]:
#Writing the clusters to file
comp_num = 0
for elem in comp:
    newpath = CLUSTERS + "\\" + str(comp_num)
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    img_num = 0
    for img in elem:
        cv2.imwrite(newpath + "\\" + str(img_num) + '.jpg', read_image(img)[1])
        img_num += 1
    comp_num += 1